### Contents:
* [Competition Introduction](#Competition-Introduction)
* [EDA](#EDA)
* [LGBM, XGB & Catboost Baseline Ensemble](#LGBM,-XGB-&-Catboost-Baseline-Ensemble)
* [Credits](#Credits)


# Competition Introduction

## Introduction to MCTS

Monte-Carlo tree search (MCTS) operates by simulating random playouts of potential game states and expanding the most promising branches of a game tree, balancing exploration of new strategies with the exploitation of known successful paths. However, MCTS comes in various variants, each tailored to specific types of games or problem spaces.

Here are some good resources for further study of MCTS:
1. [A Survey of Monte-Carlo Techniques in Games](https://www.cs.umd.edu/sites/default/files/scholarly_papers/Rajkumar_1.pdf) 
2. [Monte Carlo Tree Search: A Review of Recent Modifications and Applications](https://arxiv.org/abs/2103.04931)

## The Data

Here is the description of some prominent columns in train.csv as described in the official overview:
* **GameRulesetName** - (string) A combination of the game name and ruleset name in Ludii. Within the Ludii system, there is a distinction between games (cultural artifacts) and rulesets (the same game might be played according to different rulesets). **For the purposes of this competition, think of every unique combination of a game + a ruleset as a separate game**, although some games (ones that have many different rulesets) might be considered overrepresented in the training data.

* **EnglishRules** - (string) An natural language (English) description of the rules of the game. This description is not guaranteed to be self-contained (e.g., it might refer to rules from famous other games, such as Chess, for brevity), unambiguous, or perfectly complete.

* **LudRules** - (string) The description of the game in Ludii's game description language. This is the description that was used to compile the game inside Ludii and run the simulations, so it is always guaranteed to be 100% complete and unambiguous. However, this is a formal language that most existing Large Language Models / foundation models have likely received little, if any, exposure to.

* **utility_agent1** - (float) The **target** column. The utility value that the first agent received, aggregated over all simulations we ran for this specific pair of agents in this game. This value will be between -1 (if the first agent lost every single game) and 1 (if the first agent won every single game). Utility is calculated as (n_games_won - n_games_lost) / n_games

**test.csv** - The same as train.csv minus the following columns: num_wins_agent1, num_draws_agent1, num_losses_agent1, and utility_agent1 columns. Expect approximately 60,000 rows in the hidden test set.

**concepts.csv** - A file, exported from the publicly available Ludii database, containing information on the Concept-based features of games.

**Objective**: This competition challenges you to develop a model that can predict the performance of one MCTS variant against another in a given game, based on the features of the game.

## What's Ludii
Ludii is a general game system designed to play, evaluate and design a wide range of games, including board games, card games, dice games, mathematical games, and so on. Games are described as structured sets of ludemes (units of game-related information). This allows the full range of traditional strategy games from around the world to be modelled in a single playable database.

As a simple example, the following code shows the full game description for Tic-Tac-Toe:

`(game "Tic-Tac-Toe"
   (players 2)
   (equipment
      {
         (board (square 3))
         (piece "Disc" P1)
         (piece "Cross" P2)
      }
   )
   (rules
      (play (move Add (to (sites Empty))))
      (end (if (is Line 3) (result Mover Win)))
   )
)`


Here are some useful resources to further understand Ludii concepts:
* [Ludii Official Website](https://ludii.games/index.php)
* [Ludii Tutorials](https://ludiitutorials.readthedocs.io/en/latest/)
* [Ludeme Tree visualization](https://ludii.games/ludemeTree.php)

## Regarding the Polars Library
As the polars library (recommended here) might be less known to beginners as compared to Pandas, here is a brief introduction for those not much familiar with the Polars library :

* It provides features to read, transform & manipulate tabular data just like Pandas
* The API is very similar to Pandas & other common data wrangling libraries
* It is optimized for parallel execution on modern hardware & can achieve substantial performance gains compared to pandas in many operations
* Here is a chart showing performance comparison with other similar frameworks on the TPC-H benchmark

![](https://pola.rs/_astro/perf-illustration.jHjw6PiD_165TDG.svg)


### Basic useful functions from Polars library
* polars.DataFrame() : Create a Polars dataframe from Python Dictionary or Lists
* read_csv() & write_csv() : Read & write csv data.
* describe() : Summary statistics for a DataFrame.
* head() : Get the first n rows.
* clone() : Create a copy of this DataFrame
* count() : Return the number of non-null elements for each column.
* drop() : Remove columns from the dataframe.
* fill_null() : Fill null values using the specified value or strategy.
* filter() : Filter the rows in the DataFrame based on one or more predicate expressions.
* get_column() : Get a single column by name
* group_by() : Start a group by operation.
* select() : Select columns from this DataFrame.
* sql() : Execute a SQL query against the DataFrame.
* to_pandas() : Convert this DataFrame to a pandas DataFrame.

Further API Reference: https://docs.pola.rs/api/python/stable/reference/index.html

# EDA

In [ ]:
# Imports
import os
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
import lightgbm as lgbm
import kaggle_evaluation.mcts_inference_server
import xgboost as xgb
from catboost import CatBoostRegressor
import warnings

In [ ]:
# Reading the data
train = pl.read_csv('/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv')
train.head()

In [ ]:
train.describe()

Some Observations:
* The train data has around 233k rows & more than 800 feature columns
* Some columns seem to have a constant value across all rows as evident by same min & max values. These columns might be useless for training purpose & it would be better drop them before training any model.

## Games

Some Important Points:
* All of the games are two-player, sequential, zero-sum board games with perfect information. Your task is to predict the degree of advantage the first agent has over the other.
* There are two columns which have data about rule of games. One is "EnglishRules" and another one is "LudRules". 

In [ ]:
gameplay_counts = train[["agent1", "GameRulesetName"]].group_by("GameRulesetName").len().sort(by="len", descending=True).to_pandas()
gameplay_counts

* There are 1377 games in the training dataset. The test dataset will have other, previously unseen, games.

In [ ]:
warnings.filterwarnings("ignore", "use_inf_as_na")

plt.figure(figsize=(6, 4))
ax = sns.histplot(data=gameplay_counts, x="len", kde=True)

plt.xlabel("Number of Agent-pairs", fontsize=12)
plt.ylabel("Number of Games", fontsize=12)

plt.show()

### Distribution of the target column

In [ ]:
sns.boxplot(data=train.to_pandas(), y='utility_agent1')

* We see that most of the games have between 150 to to 200 agent-pair gameplays in the train data

## Agents (Players)

Agent String Descriptions
All agent string descriptions in training and test data are in the following format: **MCTS-\<SELECTION>-\<EXPLORATION_CONST>-\<PLAYOUT>-\<SCORE_BOUNDS>**, where:

* SELECTION : These are different strategies that may be used within the Selection phase of the MCTS algorithm. It is one of: 
    - UCB1 
    - UCB1GRAVE 
    - ProgressiveHistory
    - UCB1Tuned.
    
    
* EXPLORATION_CONST: These are three different values that have been tested for the "exploration constant" (a numeric hyperparameter shared among all of the tested Selection strategies). It is one of: 
    - 0.1 
    - 0.6
    - 1.41421356237
   
   
* PLAYOUT : These are different strategies that may be used within the Play-out phase of the MCTS algorithm. It is one of: 
    - Random200 
    - MAST
    - NST
    
    
* SCORE_BOUNDS is one of: true or false, indicating whether or not a "Score-Bounded" version of MCTS (a version of the algorithm that can prove when certain nodes in its search tree are wins/losses/draws under perfect play, and adapt the search accordingly).
    

In [ ]:
train.select(pl.col('agent1', 'agent2').n_unique())

* There are 72 unique players (agents) which we can also deduce as there 4*3*3*2=72 possible combinations of the four features

In [ ]:
plt.hist(train.select(pl.col('agent1')), bins=72, histtype='step')
plt.xticks(visible=False)
plt.xlabel('Agents (72 unique)')
plt.ylabel('Frequency as Agent1')

* From the above histogram we can see that all players play with similar frequencies.

In [ ]:
# Has any agent played against itself?
train.select((pl.col('agent1') == pl.col('agent2')).alias('Games of a player against itself')).sum()

* Thus there is no instance in the training data where the same agent played against itself

# LGBM, XGB & Catboost Baseline Ensemble

### Extracting Features From Agent Names
As suggested in the official description, we can extract four features from each agent name as follows

In [ ]:
def extract_agent_comuns(df):
    
    # Features extracted from agent names
    new_cols = df.select(pl.col('agent1').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 1).alias('p1_selection'),
                 pl.col('agent1').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 2).alias('p1_exploration').cast(pl.Float32),
                 pl.col('agent1').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 3).alias('p1_playout'),
                 pl.col('agent1').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 4).alias('p1_bounds'),
                 pl.col('agent2').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 1).alias('p2_selection'),
                 pl.col('agent2').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 2).alias('p2_exploration').cast(pl.Float32),
                 pl.col('agent2').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 3).alias('p2_playout'),
                 pl.col('agent2').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 4).alias('p2_bounds')
                )
    return df.with_columns(new_cols)

In [ ]:
target = 'utility_agent1'

X = extract_agent_comuns(train)

# Drop ID & target related columns which are not present in test data
X = X.drop(['Id', 'num_draws_agent1', 'num_losses_agent1', 'num_wins_agent1', target])

# Drop constant value columns
constant_columns = np.array(X.columns)[X.select(pl.all().n_unique() == 1).to_numpy().ravel()]
X = X.drop(list(constant_columns))

obj_cols =  X.select(pl.col(pl.String)).columns

enc = OrdinalEncoder(
    handle_unknown='use_encoded_value', unknown_value=-999, encoded_missing_value=-9999)
enc.fit(X[obj_cols])
X_transformed = enc.transform(X[obj_cols])
for e, c in enumerate(obj_cols):
    X = X.with_columns(pl.Series(c, X_transformed[:, e]))


Here we are going to train & use simple LGB & XGB models without any hyperparameter turning or other advanced stuff to serve as an easy to read, yet performant baseline code for everyone.

In [ ]:
train_cols = X.columns 

y = train.select(pl.col(target))

In [ ]:
# Train models

# Default LGB Regressor model
lgbm_reg = lgbm.LGBMRegressor()    

# Simple XGB model
xgb_reg = xgb.XGBRegressor()

cat_reg = CatBoostRegressor()

#Train LGB model
lgbm_reg.fit(X, y.to_numpy().ravel(), feature_name=train_cols)


In [ ]:
#Train XGBoost model
xgb_reg.fit(X, y.to_numpy().ravel())

In [ ]:
cat_reg.fit(X.to_pandas(), y.to_numpy().ravel())

## Feature Importance Plot

In [ ]:
lgbm.plot_importance(lgbm_reg, importance_type="gain", figsize=(7,6), title="Feature Importance (Top 20)",max_num_features=20)
plt.show()

* From the above plot, it is clear that  AdvantageP1 is the most important feature 

## Submission

In [ ]:
def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    
    # Each batch of predictions (except the very first) must be returned within 10 minutes.
    test = extract_agent_comuns(test)
    
    test_transformed = enc.transform(test[obj_cols])
    for e, c in enumerate(obj_cols):
        test = test.with_columns(pl.Series(c, test_transformed[:, e]))
        
    test = test[train_cols] 
    
    preds_lgb = lgbm_reg.predict(test.to_numpy())
    preds_xgb = xgb_reg.predict(test.to_numpy())
    preds_cat = cat_reg.predict(test.to_numpy())
    preds_mean = np.mean([preds_lgb,preds_xgb,preds_cat], axis=0)

    return sample_sub.with_columns( pl.Series('utility_agent1', preds_mean.flatten().tolist()) ) # you can use this code """pl.Series('utility_agent1', preds)"""

In [ ]:
test = pl.read_csv('/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv')
sample = pl.read_csv('/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv')
predict(test, sample)

In [ ]:
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else: 
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )


# Credits
This notebook uses some code & ideas from the following notebooks:
* https://www.kaggle.com/code/ambrosm/mcts-eda-which-makes-sense 

This notebook is a work in progress 🚧, please upvote 👍 if you find it useful & please share your feedback in the comments